In [1]:
import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)

Using cache found in /Users/wolfv/.cache/torch/hub/pytorch_vision_v0.10.0
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/wolfv/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
52.5%IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [3]:
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [12]:
from pathlib import Path
# filename = Path("~/Downloads/IMG_1234.JPG").expanduser()
filename = Path("./horse.jpg")

In [13]:
# sample execution (requires torchvision)
from PIL import Image
from torchvision import transforms
input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over ImageNet's 1000 classes
print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = torch.nn.functional.softmax(output[0], dim=0)
print(probabilities)

tensor([ 3.6909e-01, -1.6905e+00, -1.8153e+00, -1.2618e+00,  2.6069e-01,
         3.7500e-01,  4.7829e-02,  2.7368e+00,  9.0517e-01,  3.2281e+00,
        -2.4484e+00, -2.7759e+00, -2.9328e+00, -5.5310e+00, -2.8957e+00,
         8.2126e-01, -1.6317e+00, -8.3464e-01,  8.3156e-02, -3.5577e+00,
        -2.7992e+00,  2.5638e+00, -2.7399e+00,  1.7731e+00, -2.2681e+00,
        -2.4189e-01, -1.1163e+00, -2.2205e+00, -1.0008e+00,  4.5358e-02,
         1.0321e+00,  4.5819e-01,  7.0607e-01, -5.1894e-01, -5.5072e-02,
         4.6506e-01,  8.0041e-02,  1.8527e-01,  9.9622e-02, -1.1573e+00,
        -1.1511e+00, -4.8442e-01, -4.1606e-01,  9.8445e-02, -1.7513e+00,
        -1.3516e+00, -2.4572e+00,  9.5565e-01, -5.3728e-01, -3.4754e-02,
        -3.8367e+00, -2.3884e-01,  1.1949e+00, -2.0820e-01,  2.1950e-02,
        -8.5374e-02, -1.2307e+00, -2.0328e+00, -1.2342e+00, -5.3727e-01,
         7.7565e-01, -5.7994e-01, -7.0119e-01,  1.9651e+00,  1.5144e+00,
         8.2133e-01,  4.4028e-01,  5.6642e-01, -6.3

In [14]:
# Download ImageNet labels
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt


--2024-04-10 18:10:29--  https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8002::154, 2606:50c0:8000::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10472 (10K) [text/plain]
Saving to: ‘imagenet_classes.txt.1’

imagenet_classes.tx 100%[===================>]  10.23K  --.-KB/s    in 0s      

2024-04-10 18:10:29 (75.1 MB/s) - ‘imagenet_classes.txt.1’ saved [10472/10472]



In [15]:
# Read the categories
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
# Show top categories per image
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())

sorrel 0.9094085097312927
hartebeest 0.05195867642760277
impala 0.008043952286243439
Rhodesian ridgeback 0.0077136121690273285
gazelle 0.0028154540341347456
